# Configuration

In [ ]:
# Configuration #
config = {
    'validation_times': None,
    'data_name': None,  # {'breast_cancer', 'cifar10', 'MNIST', 'FashionMNIST', 'spambase', 'abalone', 'iris', 'wine'}
    'perturb_type': None,  # {None, 'mixup', 'mixup sc', 'mixup nb', 'gauss VRM'}
    'augWPL': None,  # {None, 0.25, 0.5, 0.75, 0.9, 0.99} or others
    'geometric_param': None,  # {0.25, 0.5, 0.75, 1.} or others
    'gauss_vicinal_std': None,  # {0.25, 0.5, 0.75, 1.} or others
    'batch_size': None,
    'step_size': None,  # {0.1, 0.01, 0.001, 0.0005} or others
    'epochs': None,
    'L2_decay': None,  # {0., 1e-4} or others
    'alpha': None,  # {0.25, 0.5, 0.75, 1.} or others
    'breast_spam_model': None  # {'complex_tanh_sigmoid', 'simple_relu_sigmoid', 'simple_relu_softmax'}; ONLY VALID for legacy 'breast_cancer' and 'spambase'
}
if config['data_name'] in ['breast_cancer', 'spambase']:
    if config['breast_spam_model'] in ['complex_tanh_sigmoid', 'simple_relu_sigmoid']:
        config['criterion_type'] = 'BCE'
    elif config['breast_spam_model'] in ['simple_relu_softmax']:
        config['criterion_type'] = 'CE'
elif config['data_name'] in  ['cifar10', 'MNIST', 'FashionMNIST', 'abalone', 'iris', 'wine']:
    config['criterion_type'] = 'CE'

# Folders and Files #
gdrive_dir = '/content/gdrive'
experiment_folder_dir = gdrive_dir + '/My Drive/colab'
save_folder_dir = experiment_folder_dir + '/{}-{}'.format(str(config['data_name']), str(config['perturb_type']))
file_prefix = '{}-{}-{}-{}-'.format(str(config['augWPL']), str(config['alpha']), str(config['geometric_param']), str(config['gauss_vicinal_std']))

# Libraries

In [ ]:
import numpy as np
from google.colab import drive
drive.mount(gdrive_dir)
import os
import json

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Save / Load history

In [ ]:
def load_history(filename_dir):
    with open(filename_dir, 'r') as fp:
        history = json.load(fp)
    return history

# Get Histories

In [ ]:
history_filenames = [x for x in os.listdir(save_folder_dir) if x.startswith(file_prefix) and x.endswith('.json')]
print('number of files: {}'.format(len(history_filenames)))

#  Validating the Mean of Accuracies and Losses

In [ ]:
max_test_accs = np.array([])
min_test_losses = np.array([])
max_train_accs = np.array([])
min_train_losses = np.array([])
for history_filename in history_filenames:
    history = load_history(save_folder_dir + '/' + history_filename)
    max_test_accs = np.hstack((max_test_accs, np.max(np.array(history['epoch_test_accuracy']))))
    min_test_losses = np.hstack((min_test_losses, np.min(np.array(history['epoch_mean_test_loss']))))
    max_train_accs = np.hstack((max_train_accs, np.max(np.array(history['epoch_train_accuracy']))))
    min_train_losses = np.hstack((min_train_losses, np.min(np.array(history['epoch_mean_train_loss']))))
print('mean of validated test accuracy: {}'.format(np.mean(max_test_accs)))
print('standard deviation of validated test accuracy: {}'.format(np.std(max_test_accs)))
print('mean of validated test loss: {}'.format(np.mean(min_test_losses)))
print('standard deviation of validated test loss: {}'.format(np.std(min_test_losses)))
print('mean of validated train accuracy: {}'.format(np.mean(max_train_accs)))
print('standard deviation of validated train accuracy: {}'.format(np.std(max_train_accs)))
print('mean of validated train loss: {}'.format(np.mean(min_train_losses)))
print('standard deviation of validated train loss: {}'.format(np.std(min_train_losses)))